# MyMDB Analyzer
This notebook accesses the IMDb-Scraping pipeline consisting of the webscraper, the database connector and provides a GUI for the interactive visualization.

![IMDb Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/440px-IMDB_Logo_2016.svg.png)

[Wikipedia IMDb Definition](https://en.wikipedia.org/wiki/IMDb):

IMDb (an acronym for `I`nternet `M`ovie `D`ata`b`ase) is an online database of information related to films, television series, podcasts, home videos, video games, and streaming content online – including cast, production crew and personal biographies [...]

This notebook visually explores rankings of movies and actors and 

## USAGE
Step through this notebook manually so everything is executed in order

#### OVERCOMING TECHNICAL DIFFICULTIES

In [1]:
# workarround for using event_loops in notebooks (ipykernel already uses the global non-reentrant one)
import nest_asyncio
nest_asyncio.apply()

#### Imports

In [2]:
import db
import analyze
import ui
import numpy as np
import matplotlib.pyplot as plt

connecting to db via conn_str: DRIVER=SQL Server;SERVER=localhost;PORT=1433;DATABASE=MyMDB;UID=SA;PWD=Pr0dRdyPw!


### SCRAPING
Challenges were respecting the site's scraping rules located in robots.txt, not getting timed out even though we followed through and handling those timeouts as well as handling (rather omitting a ton of) missing/format-inhomogenous data as well as asynchronous caching in python.

In [3]:
movies = []
import_ui = ui.init_ui(movies)
import_ui

5 movies acquired


In [5]:
movies = import_ui.movies

### INSERTION
This part was done purely in the SQL Server Backend, using a stored procedure for parsing the insertion data, and taking care of NaN values and the likes.

A trigger was used to recursively update the average instead of recomputation over several tables for each new insert.

In [ ]:
# insert results
for movie in movies:
    try:
        db.insert_movie(**movie)
    except Exception as e:
        print(f"issue at {movie}\n{e}")

{call insert_movie('Die Verurteilten',1994,'Drama','Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Clancy Brown,Gil Bellows,Mark Rolston,James Whitmore,Jeffrey DeMunn,Larry Brandenburg,Neil Giuntoli,Brian Libby,David Proval,Joseph Ragno,Jude Ciccolella,Paul McCrane,Renee Blaine,Scott Mann','64,86,77,73,64,56,66,101,76,75,63,74,81,87,76,62,-1,-1',9.2)}
{call insert_movie('Der Pate',1972,'Krimi','Marlon Brando,Al Pacino,James Caan,Diane Keaton,Richard S. Castellano,Robert Duvall,Sterling Hayden,John Marley,Richard Conte,Al Lettieri,Abe Vigoda,Talia Shire,Gianni Russo,John Cazale,Rudy Bond,Al Martino,Morgana King,Lenny Montana','99,83,83,77,89,92,107,115,113,95,102,77,79,87,110,95,93,97',9.2)}
{call insert_movie('The Dark Knight',2008,'Action','Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Maggie Gyllenhaal,Gary Oldman,Morgan Freeman,Monique Gabriela Curnen,Ron Dean,Cillian Murphy,Chin Han,Nestor Carbonell,Eric Roberts,Ritchie Coster,Anthony Michael Hall,Keith Szarabajka,C

In [ ]:
db.print_all_tables()

printing table actors
[(11, 'Marlon Brando', 99, Decimal('8.6')), (12, 'Al Pacino', 83, Decimal('8.7')), (13, 'James Caan', 83, Decimal('9.2')), (14, 'Diane Keaton', 77, Decimal('9.1')), (15, 'Richard S. Castellano', 89, Decimal('9.2')), (16, 'Robert Duvall', 92, Decimal('8.7')), (17, 'Sterling Hayden', 107, Decimal('8.8')), (18, 'John Marley', 115, Decimal('9.2')), (19, 'Christian Bale', 49, Decimal('8.4')), (20, 'Heath Ledger', 44, Decimal('9.0')), (21, 'Aaron Eckhart', 55, Decimal('9.0')), (22, 'Michael Caine', 90, Decimal('8.6')), (23, 'Maggie Gyllenhaal', 45, Decimal('9.0')), (24, 'Gary Oldman', 65, Decimal('8.5')), (25, 'Morgan Freeman', 86, Decimal('8.7')), (26, 'Monique Gabriela Curnen', 52, Decimal('9.0')), (27, 'Robert De Niro', 79, Decimal('8.4')), (28, 'John Cazale', 87, Decimal('8.6')), (29, 'Talia Shire', 77, Decimal('8.5')), (30, 'Lee Strasberg', 121, Decimal('9.0')), (31, 'Michael V. Gazzo', 100, Decimal('9.0')), (47, 'Elijah Wood', 42, Decimal('8.7')), (48, 'Viggo Mort

In [ ]:
# check for duplicates
"no duplicates found" if db.check_no_duplicates() else "duplicates in actors detected"

'no duplicates found'

In [ ]:

df_movies = analyze.get_movie_dataframe()
df_movies.head()

,id,title,year,genre,rating
0,2,Der Pate,1972,Krimi,9.2
1,3,The Dark Knight,2008,Action,9.0
2,4,Der Pate 2,1974,Krimi,9.0
3,7,Der Herr der Ringe: Die Rückkehr des Königs,2003,Action,8.9
4,8,Pulp Fiction,1994,Krimi,8.8


In [ ]:
df_actors = analyze.get_actors_dataframe()
df_actors.head()

,id,name,age,avg_rating
0,11,Marlon Brando,99,8.6
1,12,Al Pacino,83,8.7
2,13,James Caan,83,9.2
3,14,Diane Keaton,77,9.1
4,15,Richard S. Castellano,89,9.2


In [ ]:
df_movie_actors = analyze.get_movie_actors_dataframe()
display(df_movie_actors.head(2))
df_movie_actors.tail(3)

,movie_id,actor_id
0,2,11
1,2,12


,movie_id,actor_id
1247,447,1857
1248,447,1858
1249,447,1859


In [ ]:
df_summary = (
 analyze.get_summary_dataframe()
 .head(10)
 .sort_values(by="name")
)
df_summary

,name,age,avg_rating,movie_id,actor_id,title,year,genre,rating
1,Al Pacino,83,8.7,2,12,Der Pate,1972,Krimi,9.2
3,Diane Keaton,77,9.1,2,14,Der Pate,1972,Krimi,9.2
2,James Caan,83,9.2,2,13,Der Pate,1972,Krimi,9.2
7,John Marley,115,9.2,2,18,Der Pate,1972,Krimi,9.2
0,Marlon Brando,99,8.6,2,11,Der Pate,1972,Krimi,9.2
8,Marlon Brando,99,8.6,411,11,Apocalypse Now,1979,Drama,8.4
4,Richard S. Castellano,89,9.2,2,15,Der Pate,1972,Krimi,9.2
5,Robert Duvall,92,8.7,2,16,Der Pate,1972,Krimi,9.2
9,Robert Duvall,92,8.7,411,16,Apocalypse Now,1979,Drama,8.4
6,Sterling Hayden,107,8.8,2,17,Der Pate,1972,Krimi,9.2


In [ ]:
df_actors

,id,name,age,avg_rating
0,11,Marlon Brando,99,8.6
1,12,Al Pacino,83,8.7
2,13,James Caan,83,9.2
3,14,Diane Keaton,77,9.1
4,15,Richard S. Castellano,89,9.2
...,...,...,...,...
1007,1855,Rod Steiger,-1,8.1
1008,1856,Pat Henning,-1,8.1
1009,1857,Pete Postlethwaite,-1,8.1
1010,1858,Alison Crosbie,-1,8.1


In [ ]:
df_summary.describe().drop(["movie_id", "actor_id"], axis=1)

,age,avg_rating,year
count,10.000000,10.000000,10.000000
mean,93.600000,8.880000,1973.400000
std,11.654279,0.261619,2.951459
min,77.000000,8.600000,1972.000000
25%,84.500000,8.700000,1972.000000
50%,92.000000,8.750000,1972.000000
75%,99.000000,9.175000,1972.000000
max,115.000000,9.200000,1979.000000


In [ ]:
df_features = analyze.get_actor_feature_dataframe()
display(df_features.head())
df_features.describe()

,name,age,appearances,avg_rating
0,William Sadler,73,1,9.2
1,Tim Robbins,64,1,9.2
2,Richard S. Castellano,89,1,9.2
3,John Marley,115,1,9.2
4,James Caan,83,1,9.2


,age,appearances,avg_rating
count,613.000000,613.000000,613.000000
mean,76.482871,1.378467,8.378630
std,27.848947,0.861481,0.208911
min,18.000000,1.000000,8.000000
25%,55.000000,1.000000,8.200000
50%,73.000000,1.000000,8.300000
75%,97.000000,1.000000,8.500000
max,155.000000,9.000000,9.200000


with the scraping, inserting and loading being done let's do some fun visualizations:

# outlook

- scrape random movies and try to classify if a given actor could really make it using the top 250 (interesting outliers for certain actors included in top 250 that say only did a single movie)
- try to predict imdb rating